# **자연어 7일 - Supervised Learning**
- **True Value** 가 특정되어야 합니다
- Label Target 값들이 특정되어야 합니다
- https://ini.korea.ac.kr/colleagues/ (gon0121@naver.com : 땅콩곤두 (류기곤))

# **Naive Bayse 실습**
$ P(c|d) = P(c) 누적곱 P(t_k|c) $
1. 하지만 값이 너무 작아지므로, **Log 를 씌워서 누적합의 공식으로** 변환하여 적용

## **1 실습 데이터 만들기**
훈련데이터, 평가할 Sample 데이터 생성

In [1]:
# 스팸 필터링 기준 데이터 Set 정의하기
collection = [
    {"docID":1, "document":"Chinese Beijing Chinese", "C":"yes"},
    {"docID":2, "document":"Chinese Chinese Shanghai", "C":"yes"},
    {"docID":3, "document":"Chinese Macao", "C":"yes"},
    {"docID":4, "document":"Tokyo Japan Chinese", "C":"no"},
    {"docID":5, "document":"Chinese Chinese Chinese Tokyo Japan", "C":None},
]

trainingSet  = collection[:-1] # 훈련용 데이터 셋
sampleSet    = collection[-1]  # 스팸 여부 판단할 텍스트 셋

# V : Vocabluary (large Code : 단어 Token 의 생성)
V = []
for d in trainingSet:
    V.extend(d["document"].split())
V = list(set(V))
N = len(trainingSet)
V, N

(['Macao', 'Shanghai', 'Japan', 'Beijing', 'Tokyo', 'Chinese'], 4)

## **2 Naive Bayse 모델을 위한 데이터 생성**
이진 분류용 모델 만들기

In [2]:
C        = ["yes","no"] # 분류용 Target 값
Nc       = {}
Prior    = {}           # Liklihood 의 저장

# Conditional Probability (Zero Index)
import numpy as np
CondProb = [list(np.zeros(len(V)))  for _ in range(len(C))]
CondProb

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [3]:
# Naive Bayse 모델 만들기
from collections import defaultdict
for c in C:
    cSet     = [_ for _ in trainingSet if _["C"] == c] # 스팸 yes, no
    Nc[c]    = len(cSet)
    Prior[c] = Nc[c] / N
    
    Tc       = defaultdict(int)
    for d in cSet:
        for t in d["document"].split():
            Tc[t] += 1

    for t in V:
        CondProb[C.index(c)][V.index(t)] = \
            (Tc[t] + 1) / (sum(Tc.values()) + len(V))
        # 문장별 학습내용 살펴보기
        print(c, t, Tc[t], sum(Tc.values()), len(V))

yes Macao 1 8 6
yes Shanghai 1 8 6
yes Japan 0 8 6
yes Beijing 1 8 6
yes Tokyo 0 8 6
yes Chinese 5 8 6
no Macao 0 3 6
no Shanghai 0 3 6
no Japan 1 3 6
no Beijing 0 3 6
no Tokyo 1 3 6
no Chinese 1 3 6


## **3 Naive Bayse 모델 만들기**
이진 분류용 모델 만들기

In [4]:
# log 를 활용하여 Naive Bayse 모델로 판단하기
from math import log
score = defaultdict(float)
W     = sampleSet["document"].split()

# Log() 를 사용한 누적확률값 계산
for c in C:
    score[c] = log(Prior[c])
    for t in W:
        score[c] += log(CondProb[C.index(c)][V.index(t)])

# Naive Bayse 학습 결과값을 exp() 로 % 변환하기
from math import exp
if list(score.values())[0] > list(score.values())[1]:
    percent = exp(score["yes"]) / sum([exp(score["yes"]), exp(score["no"])])
    print("Yes 확률 {:.4} %\n {}".format(percent, dict(score)))
else:
    percent = exp(score["no"]) / sum([exp(score["yes"]), exp(score["no"])])
    print("No 확률 {:.4} %\n {}".format(percent, dict(score)))

Yes 확률 0.6898 %
 {'yes': -8.10769031284391, 'no': -8.906681345001262}


# **스팸 메일을 활용한 Naive Bayse 실습**
1. **Binary 모델링** 뿐만 아니라, **Mulity nominal** 모델링도 가능합니다

## **1 Selenium 드라이버의 활성화**
Naver 메일을 크롤링 한 뒤 스팸 필터링 성능 확인하기

In [5]:
# selenium 을 활용한 Naver 메일 접속하기
url = "https://mail.naver.com/"

# 크로니움 드라이버가 실행되면 로그인 작업을 시행 합니다
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver')
driver.get(url)

## **2 Naver Mail 의 Json 데이터를 활용하여 정보를 수집합니다**
1. requests 의 **Session() 객체를** 활용 합니다
1. **Session()** 객체를 활용하면 쉽게 **Json** 추출이 가능 합니다

In [6]:
# cookie 데이터를 저장할 Session 객체 만들고 저장하기
from requests import Session
session = Session()
for _ in driver.get_cookies():
    session.cookies.set(_["name"], _["value"])
driver.close()

In [7]:
# SN=0 : 받은메일함
# SN=5 : 스펨메일함
url_base = "https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN={}&type=&isUnread=false&u=saltman21"

# SN=0 : 일반 메일의 제목 가져오기
resp = session.post(url_base.format(0))
data1 = [_["subject"] for _ in resp.json()["mailData"]]

# SN=5 : 스펨 메일의 제목 가져오기
resp = session.post(url_base.format(5))
data2 = [_["subject"] for _ in resp.json()["mailData"]]

data1, data2

(['[수원영상미디어센터] 종강 사진',
  '헬로 데이터 과학 세미나 발표 자료 &amp; 피드백 요청',
  'SignKorea 공인인증서 갱신(기간연장) 안내',
  '[증플캠퍼스] 전일자 강의관련 추가자료',
  '[ONOFFMIX] 신기술 콘서트 설문결과 송부 및 페이스북 강연 자료 안내',
  'RE: 엑셀 VBA 3일차 강의 자료 정리',
  'RE: RE: 서울시 크리에티브 수업 필기정리',
  '9기 크리에이터 활동집 ',
  '자료',
  'Re: 오!! 잘 지내시는 지요!',
  '11월 7일 서비스디자인 교육자료 보내드립니다.',
  'Re: 김새론 실종사건 4고 최종',
  '금주 크리에이터 교육일정 관련 ',
  'PDF정리자료',
  ''],
 ['2019 이벤트 해외정품 비/아/할/인!',
  '(광고) 얼마든지 소자본으로도 창업이 가능합니다.',
  '올해 마지막, 묵혀있는 데이터 활용을 위한 상담과 솔루션을 무료로 지원받을 수 있는 방법을 공유드립니다!'])

In [8]:
# Collection 을 활용하여 수집된 정보들을 정리 합니다
collection = []
for i, _ in enumerate(data1):  # 정상 메일의 저장
    collection.append({"docID":(i+1),            "document":_, "C":"no"})
for i, _ in enumerate(data2):  # 스펨 메일의 저장
    collection.append({"docID":(len(data1)+i+1), "document":_, "C":"yes"})

collection[-5:]

[{'docID': 14, 'document': 'PDF정리자료', 'C': 'no'},
 {'docID': 15, 'document': '', 'C': 'no'},
 {'docID': 16, 'document': '2019 이벤트 해외정품 비/아/할/인!', 'C': 'yes'},
 {'docID': 17, 'document': '(광고) 얼마든지 소자본으로도 창업이 가능합니다.', 'C': 'yes'},
 {'docID': 18,
  'document': '올해 마지막, 묵혀있는 데이터 활용을 위한 상담과 솔루션을 무료로 지원받을 수 있는 방법을 공유드립니다!',
  'C': 'yes'}]

In [9]:
# 스팸과 일반 메일에 사용된 Token 목록을 생성합니다
from konlpy.tag import Mecab
ma = Mecab().morphs
V  = []
for d in collection:
    V += [_ for _ in ma(d["document"]) if len(_) > 1]
V = list(set(V))  # 문장 전
N = len(collection)
V[:7] ,N

(['영상미디어센터', '정리', 'amp', '일자', 'RE', '가능', '데이터'], 18)

## **3 정제된 Konlpy 의 Token 을 정제 합니다**
Mecab 을 활용하여, 형태소 분리가능한 Token 별로 문장요소를 나눕니다

In [10]:
# 
C     = ["yes", "no"]
Nc    = {}
Prior = {}
CondProb = [list(np.zeros(len(V)))  for _ in range(len(C))]

for c in C:
    cSet     = [_  for _ in collection   if _["C"] == c]
    Nc[c]    = len(cSet)
    Prior[c] = Nc[c]/N
    Tc       = defaultdict(int)
    for d in cSet:
        for t in [_ for _ in ma(d["document"])  if len(_) > 1]:
            Tc[t] += 1

    for t in V:
        CondProb[C.index(c)][V.index(t)] = \
            (Tc[t] + 1) / (sum(Tc.values()) + len(V))
        # 문장별 학습내용 살펴보기
        print(c, t, Tc[t], sum(Tc.values()), len(V))

yes 영상미디어센터 0 25 82
yes 정리 0 25 82
yes amp 0 25 82
yes 일자 0 25 82
yes RE 0 25 82
yes 가능 1 25 82
yes 데이터 1 25 82
yes 2019 1 25 82
yes 무료 1 25 82
yes 올해 1 25 82
yes 캠퍼스 0 25 82
yes 공인 0 25 82
yes 에티 0 25 82
yes 지원 1 25 82
yes 송부 0 25 82
yes 사진 0 25 82
yes 추가 0 25 82
yes 페이스북 0 25 82
yes 요청 0 25 82
yes PDF 0 25 82
yes 마지막 1 25 82
yes 으로 1 25 82
yes 이벤트 1 25 82
yes 과학 0 25 82
yes 증서 0 25 82
yes 세미나 0 25 82
yes SignKorea 0 25 82
yes 크리에이터 0 25 82
yes VBA 0 25 82
yes 지내 0 25 82
yes 기간 0 25 82
yes 서울시 0 25 82
yes 지요 0 25 82
yes 보내 0 25 82
yes 드립니다 1 25 82
yes 실종사건 0 25 82
yes 수원 0 25 82
yes 묵혀 1 25 82
yes 공유 1 25 82
yes 합니다 1 25 82
yes 헬로 0 25 82
yes 일정 0 25 82
yes 정품 1 25 82
yes 크리 0 25 82
yes 11 0 25 82
yes 소자본 1 25 82
yes 활동 0 25 82
yes 김새론 0 25 82
yes 솔루션 1 25 82
yes 종강 0 25 82
yes 자료 0 25 82
yes 창업 1 25 82
yes 활용 1 25 82
yes 필기 0 25 82
yes 위한 1 25 82
yes 디자인 0 25 82
yes 설문 0 25 82
yes 연장 0 25 82
yes 기술 0 25 82
yes 해외 1 25 82
yes 안내 0 25 82
yes 강연 0 25 82
yes Re 0 25 82
yes 최종 0 25 82
yes

## **4 Token 을 활용하여 Naive 모델을 만듭니다**
**Regularization :** 차이점으로는 유효한 Token 의 갯수를 제한하기 위해 1음절 이상을 대상으로 특정 합니다

In [11]:
# example : 스팸여부를 판단하는 샘플데이터
example = "4차 산업혁명 시대 보험가입 방법 안내"

from math import log
score = defaultdict(float)
W = [_ for _ in ma(example)   if len(_) > 1]

for c in C:
    score[c] = log(Prior[c])
    for t in [_  for _ in W   if _ in V]:
        try:
            score[c] += log(CondProb[C.index(c)][V.index(t)])
            print(t, CondProb[C.index(c)][V.index(t)])
        except:
            print(CondProb[C.index(c)][V.index(t)])

if list(score.values())[0] > list(score.values())[1]:
    print("Yes")
else:
    print("No")

방법 0.018691588785046728
안내 0.009345794392523364
방법 0.006369426751592357
안내 0.01910828025477707
No


## **5 모델의 성능의 평가**
SoftMax 함수를 활용하여 평가를 진행합니다

In [12]:
# 스팸여부를 Token 별 softmax 함수를 활용하여 분석결과를 출력 합니다
def softmax(idx, logProb):
    _sum = sum([exp(_) for _ in logProb])
    return exp(logProb[idx])/_sum
    
softmax(0, list(score.values()))

0.22303456949867212

In [13]:
# 스팸/ 비스팸 여부의 판단
exp(CondProb[0][V.index("디자인")]), CondProb[1][V.index("디자인")]

(1.0093896026970512, 0.012738853503184714)

In [14]:
CondProb[0][V.index("디자인")] / sum([CondProb[0][V.index("디자인")], CondProb[1][V.index("디자인")]])

0.42318059299191374